<a href="https://colab.research.google.com/github/Tamaghnatech/Claims_Severity_Allstate/blob/main/AllState_Claims_Severity_Check_using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing necessary libraries (if any)
!pip install pandas scikit-learn


In [ ]:
# Importing necessary libraries
import pandas as pd

# Paths
train_path = '/content/train.csv'
test_path = '/content/test.csv'

# Loading the datasets
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

# Checking the first few rows
print(train_data.head())


   id cat1 cat2 cat3 cat4 cat5 cat6 cat7 cat8 cat9  ...     cont6     cont7  \
0   1    A    B    A    B    A    A    A    A    B  ...  0.718367  0.335060   
1   2    A    B    A    A    A    A    A    A    B  ...  0.438917  0.436585   
2   5    A    B    A    A    B    A    A    A    B  ...  0.289648  0.315545   
3  10    B    B    A    B    A    A    A    A    B  ...  0.440945  0.391128   
4  11    A    B    A    B    A    A    A    A    B  ...  0.178193  0.247408   

     cont8    cont9   cont10    cont11    cont12    cont13    cont14     loss  
0  0.30260  0.67135  0.83510  0.569745  0.594646  0.822493  0.714843  2213.18  
1  0.60087  0.35127  0.43919  0.338312  0.366307  0.611431  0.304496  1283.60  
2  0.27320  0.26076  0.32446  0.381398  0.373424  0.195709  0.774425  3005.09  
3  0.31796  0.32128  0.44467  0.327915  0.321570  0.605077  0.602642   939.85  
4  0.24564  0.22089  0.21230  0.204687  0.202213  0.246011  0.432606  2763.85  

[5 rows x 132 columns]


In [ ]:
# Splitting features and target
X = train_data.drop(columns=['id', 'loss'])
y = train_data['loss']

# One hot encoding the categorical features
X_encoded = pd.get_dummies(X, drop_first=True)

# Do the same for the test dataset
test_ids = test_data['id']
test_data_encoded = pd.get_dummies(test_data.drop('id', axis=1), drop_first=True)


I used Random Forest but unfortunately it wasnt effective enough on the training data so I used XG boost

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Splitting the dataset for validation
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Training the Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [ ]:
# Predictions on the validation set
y_pred = rf.predict(X_val)

# Calculating RMSE
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"RMSE: {rmse}")

RMSE: 1918.2983110646376


In [ ]:
!pip install xgboost


In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Convert the dataset into an optimized data structure that the XGBoost library provides.
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# Set parameters
param = {
    'max_depth': 6,  # Maximum depth of a tree
    'eta': 0.3,  # Step size shrinkage
    'objective': 'reg:squarederror',  # Regression with squared error
    'eval_metric': 'rmse',  # Root mean squared error as evaluation metric
}
epochs = 100

# Train the model
bst = xgb.train(param, dtrain, epochs, [(dtrain, 'train'), (dval, 'eval')], early_stopping_rounds=10, verbose_eval=10)

# Predicting on the validation set
y_pred_val = bst.predict(dval)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"Validation RMSE: {rmse_val}")


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:617: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[0]	train-rmse:3338.57114	eval-rmse:3294.45479
[10]	train-rmse:1903.29758	eval-rmse:1944.90390
[20]	train-rmse:1812.73107	eval-rmse:1886.37360
[30]	train-rmse:1766.04404	eval-rmse:1870.18722
[40]	train-rmse:1740.41959	eval-rmse:1864.99925
[50]	train-rmse:1715.93001	eval-rmse:1863.24278
[60]	train-rmse:1702.64761	eval-rmse:1862.64158
[70]	train-rmse:1688.26526	eval-rmse:1861.73968
[80]	train-rmse:1676.07386	eval-rmse:1861.93621
[81]	train-rmse:1675.31470	eval-rmse:1862.02961
Validation RMSE: 1862.0296098167205


This is Hyper Paramter Tuning for XG Boost, a little warning that is this step is very GPU intensive so it wont work if you dont have a good amount of RAM and a powerful GPU supporting it.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb

# Creating a XGBoost Regressor estimator
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror')

# Defining the hyperparameter distributions (notice some are now lists, others can be distributions)
param_dist = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [4, 6, 8],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Randomized search with 5-fold cross-validation
random_search = RandomizedSearchCV(estimator=xgb_reg, param_distributions=param_dist,
                                   n_iter=10, # Number of parameter settings that are sampled
                                   scoring='neg_mean_squared_error', cv=5, verbose=2, n_jobs=-1, random_state=42)

random_search.fit(X_encoded, y)

# Getting the best parameters
best_params = random_search.best_params_
print(f"Best parameters: {best_params}")

# Using the best model
best_xgb = random_search.best_estimator_


In [ ]:
all_data = pd.concat([X_train, X_val, test_data.drop('id', axis=1)], axis=0)
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(all_data)


OneHotEncoder(handle_unknown='ignore')

In [ ]:
print(all_data.columns)
print(X_train.columns)


Index(['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8',
       'cont9', 'cont10',
       ...
       'cat107', 'cat108', 'cat109', 'cat110', 'cat111', 'cat112', 'cat113',
       'cat114', 'cat115', 'cat116'],
      dtype='object', length=1153)
Index(['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8',
       'cont9', 'cont10',
       ...
       'cat116_P', 'cat116_Q', 'cat116_R', 'cat116_S', 'cat116_T', 'cat116_U',
       'cat116_V', 'cat116_W', 'cat116_X', 'cat116_Y'],
      dtype='object', length=1037)


In [ ]:
# Select only the categorical columns for encoding
categorical_features = all_data.select_dtypes(include=['object']).columns

# Fit the encoder
encoder.fit(all_data[categorical_features])
